In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [3]:
import configparser
import sys
import ctypes
import argparse
import math
import numpy as np
import tensorrt as trt
import torch
from datetime import datetime

from transformers import PreTrainedTokenizerFast
from transformers import T5Tokenizer, T5TokenizerFast

# dir_path = os.path.dirname(os.path.realpath(__file__))
sys.path.append("/..")
# from examples.pytorch.decoding.utils.recover_bpe import recover_bpe

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def recover_bpe(src):
    dst = []
    for line in src:
        line = line.strip().split()
        if line[-1] == '</s>':
            line.pop()
        if line[0][0] == '▁':
            s = line[0][1:]
        else:
            s = line[0]
        for w in line[1:]:
            if w[0] == '▁':
                s += ' ' + w[1:]
            else:
                s += w
        s += '\n'
        dst.append(s)
    return dst

In [5]:
def bleu_score(pred, ref):
    from sacrebleu import corpus_bleu
    bleu = corpus_bleu(pred, [ref], force=True)
    print("       bleu score: {:6.2f}".format(bleu.score))
    print("       bleu counts: {}".format(bleu.counts))
    print("       bleu totals: {}".format(bleu.totals))
    print("       bleu precisions: {}".format(bleu.precisions))
    print("       bleu sys_len: {}; ref_len: {}".format(bleu.sys_len, bleu.ref_len))
    return bleu

In [6]:
npToTrt = {np.int8: trt.int8, np.float16: trt.float16, np.int32: trt.int32, np.float32: trt.float32}
npToPFT = {np.int8: trt.PluginFieldType.INT8, np.float16: trt.PluginFieldType.FLOAT16, np.int32: trt.PluginFieldType.INT32, np.float32: trt.PluginFieldType.FLOAT32}
npToTorch = {np.dtype('float16'): torch.float16, np.dtype('int32'): torch.int32, np.dtype('float32'): torch.float32}
device = 0

# global variables with default value
globalNMaxBatchSize = 128
globalNMaxSeqLen = 512
globalNBeamSize = 4
globalNUseFP16 = 0

globalNSM = (lambda x: x[0] * 10 + x[1])(torch.cuda.get_device_capability())

globalFBeamDiversity = 0.0
globalFTemperature = 1.0
globalFLenPenalty = 0.0
globalFRepPenalty = 1.0

nMinBatchSize = 1
nOptBatchSize = globalNMaxBatchSize
nMaxBatchSize = globalNMaxBatchSize
nMinSeqLen = 1
nOptSeqLen = globalNMaxSeqLen
nMaxSeqLen = globalNMaxSeqLen

In [7]:
def getT5EncoderPlugin(arg):
    nBatchSize = arg['batch_size']
    nMaxSeqLen = arg['max_seq_len']
    nBeamSize = arg['beam_width'],
    nSM = globalNSM
    useFP16 = int(arg['data_type'] == 'fp16')
    ckpt_path = arg['ckpt_path'].encode()
    for c in trt.get_plugin_registry().plugin_creator_list:
        if c.name == 'T5EncoderPlugin':
            pList = [
                trt.PluginField('max_batch_size', np.int32(nBatchSize), npToPFT[np.int32]),
                trt.PluginField('max_seq_len', np.int32(nMaxSeqLen), npToPFT[np.int32]),
                trt.PluginField('beam_width', np.int32(nBeamSize), npToPFT[np.int32]),
                trt.PluginField('sm', np.int32(nSM), npToPFT[np.int32]),
                trt.PluginField('useFP16', np.int32(useFP16), npToPFT[np.int32]),
                trt.PluginField('ckpt_path', ckpt_path, trt.PluginFieldType.CHAR),
            ]
            return c.create_plugin(c.name, trt.PluginFieldCollection(pList))
    return None

In [8]:
def getT5DecodingPlugin(arg):
    nBatchSize = arg['batch_size']
    nMaxSeqLen = arg['max_seq_len']
    nMemMaxSeqLen = arg['max_seq_len']
    nBeamSize = arg['beam_width']
    useFP16 = int(arg['data_type'] == 'fp16')
    ckpt_path = arg['ckpt_path'].encode()
    for c in trt.get_plugin_registry().plugin_creator_list:
        if c.name == 'T5DecodingPlugin':
            pList = [
                trt.PluginField('max_batch_size', np.int32(nBatchSize), npToPFT[np.int32]),
                trt.PluginField('max_seq_len', np.int32(nMaxSeqLen), npToPFT[np.int32]),
                trt.PluginField('mem_max_seq_len', np.int32(nMaxSeqLen), npToPFT[np.int32]),
                trt.PluginField('beam_width', np.int32(nBeamSize), npToPFT[np.int32]),
                trt.PluginField('useFP16', np.int32(useFP16), npToPFT[np.int32]),
                trt.PluginField('ckpt_path', ckpt_path, trt.PluginFieldType.CHAR),
            ]
            return c.create_plugin(c.name, trt.PluginFieldCollection(pList))
    return None

In [9]:
def buildEngine(logger, arg, trtFileName):
    builder = trt.Builder(logger)
    network = builder.create_network(1)
    profile = builder.create_optimization_profile()
    config = builder.create_builder_config()
    config.max_workspace_size = 1 << 30
    config.flags = int(arg['data_type'] == 'fp16')

    inputT0 = network.add_input('inputId', npToTrt[np.int32], [-1, -1])
    inputT1 = network.add_input('inputSeqLen', npToTrt[np.int32], [-1])
    inputT2 = network.add_input('inputTopK', npToTrt[np.int32], [-1])
    inputT3 = network.add_input('inputTopP', npToTrt[np.int32], [-1])
    inputT4 = network.add_input('inputBeam_search_diversity_rate', npToTrt[np.float32], [-1])
    inputT5 = network.add_input('inputTemperature', npToTrt[np.float32], [-1])
    inputT6 = network.add_input('inputLen_penalty', npToTrt[np.float32], [-1])
    inputT7 = network.add_input('inputRepetition_penalty', npToTrt[np.float32], [-1])

    profile.set_shape(inputT0.name, [nMinBatchSize, nMinSeqLen], [nOptBatchSize, nOptSeqLen], [nMaxBatchSize, nMaxSeqLen])
    profile.set_shape(inputT1.name, [nMinBatchSize], [nOptBatchSize], [nMaxBatchSize])
    profile.set_shape(inputT2.name, [1], [nOptBatchSize], [nMaxBatchSize])
    profile.set_shape(inputT3.name, [1], [nOptBatchSize], [nMaxBatchSize])
    profile.set_shape(inputT4.name, [1], [nOptBatchSize], [nMaxBatchSize])
    profile.set_shape(inputT5.name, [1], [nOptBatchSize], [nMaxBatchSize])
    profile.set_shape(inputT6.name, [1], [nOptBatchSize], [nMaxBatchSize])
    profile.set_shape(inputT7.name, [1], [nOptBatchSize], [nMaxBatchSize])
    config.add_optimization_profile(profile)

    model_config = configparser.ConfigParser()
    model_config_path = os.path.join(arg["ckpt_path"], 'config.ini')
    if os.path.isfile(model_config_path):
        model_config.read(model_config_path)

    encoderPlugin = getT5EncoderPlugin(arg)
    decodingPlugin = getT5DecodingPlugin(arg)
    if encoderPlugin == None:
        print("Failed making encoder plugin!")
        return None
    if decodingPlugin == None:
        print("Failed making decoding plugin!")
        return None

    encoderLayer = network.add_plugin_v2([inputT0, inputT1], encoderPlugin)
    decodingLayer = network.add_plugin_v2([encoderLayer.get_output(0), inputT1, inputT2, inputT3, inputT4, inputT5, inputT6, inputT7], decodingPlugin)
    decodingLayer.get_output(0).name = "decodingOutput0"
    decodingLayer.get_output(1).name = "decodingOutput1"
    decodingLayer.get_output(0).dtype = npToTrt[np.int32]
    decodingLayer.get_output(1).dtype = npToTrt[np.int32]

    network.mark_output(decodingLayer.get_output(0))
    network.mark_output(decodingLayer.get_output(1))

    engineString = builder.build_serialized_network(network, config)
    if engineString == None:
        print("Failed getting serialized engine!")
        return None
    print("Succeeded getting serialized engine!")
    with open(trtFileName, "wb") as f:
        f.write(engineString)
        print("Succeeded saving .plan file!")
    engine = trt.Runtime(logger).deserialize_cuda_engine(engineString)
    return engine

In [10]:
def testone(arg, stream, input_text):
    useFP16 = int(arg['data_type'] == 'fp16')

    nBatchSize = arg['batch_size']
    nSeqLen = arg['max_seq_len']
    testCase = "<fp%s,bs=%d,sl=%d>" % (['32', '16'][useFP16], nBatchSize, nSeqLen)
    print("Test both Encoder and Decoding", testCase)
    logger = trt.Logger(trt.Logger.ERROR)
    trt.init_libnvinfer_plugins(logger, '')
 
    ctypes.cdll.LoadLibrary(arg['lib_path'])

    trtFile = 'T5Engine-fp' + ['32', '16'][useFP16] + '.plan'

    if os.path.isfile(trtFile):
        with open(trtFile, 'rb') as f:
            engineString = f.read()
            engine = trt.Runtime(logger).deserialize_cuda_engine(engineString)
        if engine == None:
            print("Failed loading engine!")
            return
        print("Succeeded loading engine!")
    else:
        engine = buildEngine(logger, arg, trtFile)

    context = engine.create_execution_context()
    nInput = np.sum([engine.binding_is_input(i) for i in range(engine.num_bindings)])
    nOutput = engine.num_bindings - nInput
#     for i in range(engine.num_bindings):
#         print("Bind[%2d]:i[%d]->"%(i,i) if engine.binding_is_input(i) else "Bind[%2d]:o[%d]->"%(i,i-nInput),
#                engine.get_binding_dtype(i),engine.get_binding_shape(i),context.get_binding_shape(i),engine.get_binding_name(i))

    tokenizer = T5Tokenizer.from_pretrained(arg['model'])
    fast_tokenizer = T5TokenizerFast.from_pretrained(arg['model'])

#     with open(arg['source'], 'r') as f:
#         src_text = recover_bpe(f.readlines())
#         src_text = ["translate English to German: " + line.strip() for line in src_text]

#     with open(arg['target'], 'r') as f:
#         tgt_text = recover_bpe(f.readlines())

#     sys.stdout.flush()

    outputId = []
    outputSeqLen = []
    prev = 0
    needWarmUp = True
    torch.cuda.synchronize()
    start_time = datetime.now()

    input_texts = input_text


    input_token = tokenizer(input_texts, return_tensors='pt', padding=False)

    inputId = np.ascontiguousarray(input_token['input_ids'].numpy().astype(np.int32))
    inputMask = np.ascontiguousarray(np.sum(input_token['attention_mask'].numpy(), 1).astype(np.int32))
    nRealBatchSize, nRealSeqLen = np.shape(inputId)

    context.set_binding_shape(0, [nRealBatchSize, nRealSeqLen])
    context.set_binding_shape(1, [nRealBatchSize])
    context.set_binding_shape(2, [nRealBatchSize])
    context.set_binding_shape(3, [nRealBatchSize])
    context.set_binding_shape(4, [nRealBatchSize])
    context.set_binding_shape(5, [nRealBatchSize])
    context.set_binding_shape(6, [nRealBatchSize])
    context.set_binding_shape(7, [nRealBatchSize])

    inputTopK = np.full([nRealBatchSize], arg['sampling_topk'], dtype=np.int32)
    inputTopP = np.full([nRealBatchSize], arg['sampling_topp'], dtype=np.float32)

    inputFBeamDiversity = np.full([nRealBatchSize], globalFBeamDiversity, dtype=np.float32)
    inputFTemperature = np.full([nRealBatchSize], globalFTemperature, dtype=np.float32)
    inputFLenPenalty = np.full([nRealBatchSize], globalFLenPenalty, dtype=np.float32)
    inputFRepPenalty = np.full([nRealBatchSize], globalFRepPenalty, dtype=np.float32)

    bufferD = []
    bufferD.append(torch.from_numpy(inputId).to(device))
    bufferD.append(torch.from_numpy(inputMask).to(device))
    bufferD.append(torch.from_numpy(inputTopK).to(device))
    bufferD.append(torch.from_numpy(inputTopP).to(device))
    bufferD.append(torch.from_numpy(inputFBeamDiversity).to(device))
    bufferD.append(torch.from_numpy(inputFTemperature).to(device))
    bufferD.append(torch.from_numpy(inputFLenPenalty).to(device))
    bufferD.append(torch.from_numpy(inputFRepPenalty).to(device))
    bufferD.append(torch.empty(tuple(context.get_binding_shape(8)), dtype=torch.int32, device=device))
    bufferD.append(torch.empty(tuple(context.get_binding_shape(9)), dtype=torch.int32, device=device))
    torch.cuda.synchronize()


    context.execute_async_v2([b.data_ptr() for b in bufferD], stream)

    torch.cuda.synchronize()

    outputId.append(bufferD[nInput + 0].cpu().numpy())
    outputSeqLen.append(bufferD[nInput + 1].cpu().numpy())
    

    stop_time = datetime.now()
    execution_time = (stop_time - start_time).total_seconds()

    outputText = []
    for batch_token, batch_seq_len in zip(outputId, outputSeqLen):
        for j in range(len(batch_token)):
            outputText.append(fast_tokenizer.decode(batch_token[j][0][:batch_seq_len[j][0]], skip_special_tokens=True))
    
    print(outputText)

In [11]:
np.set_printoptions(precision=4, linewidth=200, suppress=True)

In [12]:
torch.cuda.set_device(device)

In [13]:
stream = 0
# os.system('rm -f ./*.plan ./*.in')
# parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
# parser.add_argument('-batch', '--batch_size', type=int, metavar='NUMBER', default=32, help='batch size (default: 32)')
# parser.add_argument('-beam', '--beam_width', type=int, metavar='NUMBER', default=4, help='beam width (default: 4)')
# parser.add_argument('-s', '--max_seq_len', type=int, metavar='NUMBER', default=128, help='max sequence length (default: 200)')
# parser.add_argument('--source', type=str, metavar='STRING', default="../examples/pytorch/decoding/utils/translation/test.en", help="Path to the source file.")
# parser.add_argument('--target', type=str, metavar='STRING', default="../examples/pytorch/decoding/utils/translation/test.de", help="Path to the target file.")
# parser.add_argument('-diversity_rate', '--beam_search_diversity_rate', type=float, metavar='NUMBER', default=0.0, help='deviersity rate of beam search. default is 0. When diversity rate = 0, it is equivalent to the naive beam search.')
# parser.add_argument('-topk', '--sampling_topk', type=int, metavar='NUMBER', default=4, help='Candidate (k) value of top k sampling in decoding. Default is 1.')
# parser.add_argument('-topp', '--sampling_topp', type=float, metavar='NUMBER', default=0.0, help='Probability (p) value of top p sampling in decoding. Default is 0.0. ')
# parser.add_argument('-d', '--data_type', type=str, metavar='STRING', default="fp32", help='data type (default: fp32)', choices=['fp32', 'fp16'])
# parser.add_argument('-lib_path', '--lib_path', type=str, metavar='STRING', default="lib/libtrt_t5.so", help='the path of FasterTransformer pytorch t5 op library.')
# parser.add_argument('-model', '--model', type=str, metavar='STRING', default="t5-small", help='T5 model size.', choices=["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"])
# parser.add_argument(            '--ckpt_path',      type=str, metavar='STRING', help='path to the checkpoint file.')
# parser.add_argument('-max_ite', '--max_iteration',  type=int,   metavar='NUMBER',   default=100000,     help='Maximum iteraiton for translation, default is 100000 (as large as possible to run all test set).')
# parser.add_argument('--ft_BLEU_threshold', type=float, help='Threshold of FT BLEU score')

In [14]:
arg = {}
arg["batch_size"] = 1
arg["beam_width"] = 4
arg["max_seq_len"] = globalNMaxSeqLen
arg["source"] = "../examples/pytorch/decoding/utils/translation/test.en"
arg["target"] = "../examples/pytorch/decoding/utils/translation/test.de"
arg["beam_search_diversity_rate"] = 0.0
arg["sampling_topk"] = 10
arg["sampling_topp"] = 0.0
arg["data_type"] = "fp32"
arg["lib_path"] = "lib/libtrt_t5.so"
arg["model"] = "../qa-t5-large"
arg["ckpt_path"] = "./ft_t5_large/1-gpu"
# arg["model"] = "t5-large"
# arg["ckpt_path"] = "./ft_t5_small/1-gpu"
arg["max_iteration"] = 100000
# arg["ft_BLEU_threshold"] = 

In [15]:
input_text = """### Instruction: Tạo 5 câu hỏi và trả lời cho đoạn văn này.:
Ngăn ngừa ung thư
Chất lycopene trong dưa hấu có thể làm giảm mức độ phát triển của ung thư. Vì đây là chất chống oxy hóa mạnh mẽ nên có tác dụng ngăn ngừa sự tấn công của một số căn bệnh ung thư.
Ngoài ra lycopene đặc biệt hữu ích trong điều trị nhiễm trùng HPV - nguyên nhân gây ra ung thư cổ tử cung ở phái nữ. Sử dụng một chén dưa hấu tương đương với hai cốc chứa khoảng 20 miligam lycopene để mang lại hiệu quả tối ưu.
Có thể giúp giảm đau nhức cơ bắp
Đây là loại trái cây có nhiều chất điện giải và axit amin citrulline mang lại tác dụng giúp làm dịu cơ bắp bị đau sau khi tập luyện nặng. Theo một nghiên cứu từ Iran, thành phần citrulline trong dưa hấu có thể giúp giảm đau nhức, mỏi cơ bắp nhờ vào đẩy nhanh quá trình loại bỏ axit lactic. Bên cạnh đó uống nhiều nước ép dưa hấu giúp cơ bắp của bạn nhận được nhiều oxy hơn góp phần hỗ trợ phục hồi được nhanh hơn.
Hỗ trợ tiêu hóa
Dưa hấu là một trong những loại thực phẩm tác dụng hỗ trợ cho hệ tiêu hóa khỏe mạnh nhờ vào hàm lượng nước trong loại hoa quả này. Dưa hấu còn chứa chất xơ thúc đẩy quá trình tiêu hóa và ngăn ngừa táo bón hiệu quả.
### Response: """

In [16]:
# input_text = 'Translate to German: hello my name is Khang.'

In [17]:
print(input_text)

### Instruction: Tạo 5 câu hỏi và trả lời cho đoạn văn này.:
Ngăn ngừa ung thư
Chất lycopene trong dưa hấu có thể làm giảm mức độ phát triển của ung thư. Vì đây là chất chống oxy hóa mạnh mẽ nên có tác dụng ngăn ngừa sự tấn công của một số căn bệnh ung thư.
Ngoài ra lycopene đặc biệt hữu ích trong điều trị nhiễm trùng HPV - nguyên nhân gây ra ung thư cổ tử cung ở phái nữ. Sử dụng một chén dưa hấu tương đương với hai cốc chứa khoảng 20 miligam lycopene để mang lại hiệu quả tối ưu.
Có thể giúp giảm đau nhức cơ bắp
Đây là loại trái cây có nhiều chất điện giải và axit amin citrulline mang lại tác dụng giúp làm dịu cơ bắp bị đau sau khi tập luyện nặng. Theo một nghiên cứu từ Iran, thành phần citrulline trong dưa hấu có thể giúp giảm đau nhức, mỏi cơ bắp nhờ vào đẩy nhanh quá trình loại bỏ axit lactic. Bên cạnh đó uống nhiều nước ép dưa hấu giúp cơ bắp của bạn nhận được nhiều oxy hơn góp phần hỗ trợ phục hồi được nhanh hơn.
Hỗ trợ tiêu hóa
Dưa hấu là một trong những loại thực phẩm tác dụng h

In [18]:
# testone(arg, stream, input_text)
print("Test finish!")

Test finish!


In [19]:
# !./bin/t5_gemm 1 4 128 1024 16 64 2816 1024 16 64 2816 106612 0 1 0

# Test trt

In [20]:
useFP16 = int(arg['data_type'] == 'fp16')

nBatchSize = arg['batch_size']
nSeqLen = arg['max_seq_len']
testCase = "<fp%s,bs=%d,sl=%d>" % (['32', '16'][useFP16], nBatchSize, nSeqLen)
print("Test both Encoder and Decoding", testCase)
logger = trt.Logger(trt.Logger.ERROR)
trt.init_libnvinfer_plugins(logger, '')

ctypes.cdll.LoadLibrary(arg['lib_path'])

trtFile = 'T5Engine-fp' + ['32', '16'][useFP16] + '.plan'

if os.path.isfile(trtFile):
    with open(trtFile, 'rb') as f:
        engineString = f.read()
        engine = trt.Runtime(logger).deserialize_cuda_engine(engineString)
    if engine == None:
        print("Failed loading engine!")
    print("Succeeded loading engine!")
else:
    engine = buildEngine(logger, arg, trtFile)

Test both Encoder and Decoding <fp32,bs=1,sl=512>
Succeeded getting serialized engine!
Succeeded saving .plan file!


In [21]:
fast_tokenizer = T5TokenizerFast.from_pretrained(arg['model'])

In [22]:
tokenizer = T5Tokenizer.from_pretrained(arg['model'])

In [23]:
context = engine.create_execution_context()
nInput = np.sum([engine.binding_is_input(i) for i in range(engine.num_bindings)])
nOutput = engine.num_bindings - nInput

In [27]:
input_text = """### Instruction: Tạo 5 câu hỏi và trả lời cho đoạn văn này.:
Tại sự kiện, người cao tuổi được đo mức loãng xương, huyết áp để có những phán đoán chính xác về tình trạng sức khỏe. Dựa trên kết quả thăm khám, các bác sĩ của Viện Nghiên cứu Dinh dưỡng TP.HCM (NRI) đưa ra lời khuyên bổ ích để giúp các cụ xây dựng chế độ dinh dưỡng phù hợp hơn, từ đó có thể tự chủ động chăm sóc bản thân, sống vui khỏe bên gia đình và bạn bè.
Värna tổ chức tư vấn sức khỏe và dinh dưỡng cho 1.000 người cao tuổi - 2
Nhận được tư vấn rất hữu ích, cụ Nguyễn Hoàng Diệu (70 tuổi) cho biết: “Bình thường, tôi không kiểm tra sức khỏe nên lúc đo và phát hiện bị loãng xương, tôi rất bất ngờ và cảm thấy lo lắng lắm. Tuy nhiên, nhờ sự giải thích và trấn an từ các bác sĩ, tôi đã an tâm phần nào. Những thay đổi nhỏ trong khẩu phần ăn sẽ cải thiện tình hình của tôi”.
### Response: """

In [28]:
outputId = []
outputSeqLen = []

torch.cuda.synchronize()
start_time = datetime.now()

input_texts = input_text

input_token = tokenizer(input_texts, return_tensors='pt', padding=False, max_length=globalNMaxSeqLen, truncation=True)
print(input_token['input_ids'].shape)
inputId = np.ascontiguousarray(input_token['input_ids'].numpy().astype(np.int32))
inputMask = np.ascontiguousarray(np.sum(input_token['attention_mask'].numpy(), 1).astype(np.int32))
nRealBatchSize, nRealSeqLen = np.shape(inputId)

context.set_binding_shape(0, [nRealBatchSize, nRealSeqLen])
context.set_binding_shape(1, [nRealBatchSize])
context.set_binding_shape(2, [nRealBatchSize])
context.set_binding_shape(3, [nRealBatchSize])
context.set_binding_shape(4, [nRealBatchSize])
context.set_binding_shape(5, [nRealBatchSize])
context.set_binding_shape(6, [nRealBatchSize])
context.set_binding_shape(7, [nRealBatchSize])

inputTopK = np.full([nRealBatchSize], arg['sampling_topk'], dtype=np.int32)
inputTopP = np.full([nRealBatchSize], arg['sampling_topp'], dtype=np.float32)

inputFBeamDiversity = np.full([nRealBatchSize], globalFBeamDiversity, dtype=np.float32)
inputFTemperature = np.full([nRealBatchSize], globalFTemperature, dtype=np.float32)
inputFLenPenalty = np.full([nRealBatchSize], globalFLenPenalty, dtype=np.float32)
inputFRepPenalty = np.full([nRealBatchSize], globalFRepPenalty, dtype=np.float32)

bufferD = []
bufferD.append(torch.from_numpy(inputId).to(device))
bufferD.append(torch.from_numpy(inputMask).to(device))
bufferD.append(torch.from_numpy(inputTopK).to(device))
bufferD.append(torch.from_numpy(inputTopP).to(device))
bufferD.append(torch.from_numpy(inputFBeamDiversity).to(device))
bufferD.append(torch.from_numpy(inputFTemperature).to(device))
bufferD.append(torch.from_numpy(inputFLenPenalty).to(device))
bufferD.append(torch.from_numpy(inputFRepPenalty).to(device))
bufferD.append(torch.empty(tuple(context.get_binding_shape(8)), dtype=torch.int32, device=device))
bufferD.append(torch.empty(tuple(context.get_binding_shape(9)), dtype=torch.int32, device=device))
torch.cuda.synchronize()

context.execute_async_v2([b.data_ptr() for b in bufferD], stream)

torch.cuda.synchronize()

outputId.append(bufferD[nInput + 0].cpu().numpy())
outputSeqLen.append(bufferD[nInput + 1].cpu().numpy())


stop_time = datetime.now()
execution_time = (stop_time - start_time).total_seconds()

outputText = []
for batch_token, batch_seq_len in zip(outputId, outputSeqLen):
    for j in range(len(batch_token)):
        for beam_index in range(arg["beam_width"]):
            outputText.append(fast_tokenizer.decode(batch_token[j][beam_index][:batch_seq_len[j][beam_index]], skip_special_tokens=True))
            print(batch_token[j][beam_index][:batch_seq_len[j][beam_index]].shape[0])
            print(batch_token[j][beam_index][:batch_seq_len[j][beam_index]].shape[0]/execution_time)
            print(outputText[-1])
            print("="*30)

torch.Size([1, 231])
512
160.3901490375338
-Câu hỏi 1: Người cao tuổi đã trải qua hoạt động tư vấn sức khỏe và dinh dưỡng như thế nào?, -Trả lời 1: Người cao tuổi đã trải qua hoạt động tư vấn sức khỏe và dinh dưỡng dựa trên kết quả thăm khám và tư vấn dinh dưỡng dựa trên kết quả thăm khám để xây dựng chế độ dinh dưỡng phù hợp, từ đó có thể tự chủ động chăm sóc bản thân, sống vui khỏe bên gia đình và bạn bè. -Câu hỏi 2: Người cao tuổi đã trải qua hoạt động tư vấn sức khỏe và dinh dưỡng như thế nào?, -Trả lời 2: Người cao tuổi đã trải qua hoạt động tư vấn sức khỏe và dinh dưỡng dựa trên kết quả thăm khám và tư vấn dinh dưỡng để xây dựng chế độ dinh dưỡng phù hợp, từ đó có thể tự chủ động chăm sóc bản thân, sống vui khỏe bên gia đình và bạn bè, đồng thời giúp xây dựng chế độ dinh dưỡng phù hợp để tự chủ động chăm sóc bản thân, sống vui khỏe bên gia đình và bạn bè, cũng như cải thiện tình trạng sức khỏe của họ. -Câu hỏi 3: Người cao tuổi đã trải qua hoạt động tư vấn sức khỏe và dinh dưỡng 

In [26]:
execution_time

3.159864